# 🗺️ 네이버 플레이스 크롤링 앱 (구글 코랩 버전)

## 📌 사용 방법
1. **모든 셀을 순서대로 실행**하세요 (Shift + Enter)
2. 마지막 셀 실행 후 **ngrok URL**로 접속
3. 웹 인터페이스에서 검색 & CSV 저장

## ✨ 기능
- ✅ 실제 네이버 플레이스 데이터 크롤링
- ✅ 타지역업체 자동 감지 (070번호, 동/구 주소)
- ✅ 플레이스 탭 감지
- ✅ 일괄 키워드 검증
- ✅ CSV 다운로드

---

## 1️⃣ 필요한 패키지 설치

Flask, Playwright, pyngrok 설치 (약 2-3분 소요)

In [ ]:
%%capture
!pip install flask playwright pyngrok nest-asyncio
!playwright install chromium
!playwright install-deps chromium

print("✅ 모든 패키지 설치 완료!")

## 2️⃣ ngrok 설정

외부에서 접속할 수 있도록 ngrok 터널 생성

💡 **무료 ngrok 계정 (선택사항)**:
- https://ngrok.com 에서 가입
- 토큰을 받아서 아래에 입력하면 더 안정적으로 사용 가능
- 없어도 작동하지만 2시간 제한이 있습니다

In [ ]:
# 선택사항: ngrok 토큰이 있으면 입력 (없으면 빈 문자열)
NGROK_TOKEN = ""  # 예: "2abc3def4ghi5jkl6mno7pqr8stu9vwx"

if NGROK_TOKEN:
    !ngrok authtoken {NGROK_TOKEN}
    print("✅ ngrok 토큰 설정 완료")
else:
    print("⚠️ ngrok 토큰 없이 실행 (2시간 제한)")

## 3️⃣ 크롤링 모듈 생성

실제 네이버 플레이스 크롤링을 위한 코드 생성

In [ ]:
%%writefile naver_place_crawler_real.py
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
네이버 플레이스 실제 크롤링 모듈 (코랩 최적화)
"""

from playwright.sync_api import sync_playwright
import time
import re
from urllib.parse import quote

class RealNaverPlaceCrawler:
    def __init__(self, headless=True):
        self.headless = headless
        self.playwright = None
        self.browser = None
        self.page = None
        
        self.place_keywords = [
            '맛집', '카페', '병원', '약국', '편의점', '음식점', '레스토랑',
            '미용실', '네일샵', '학원', '헬스장', '피트니스', '정형외과',
            '치과', '피부과', '안과', 'PC방', '노래방', '찜질방', '숙박',
            '호텔', '모텔', '게스트하우스', '빵집', '제과점', '분식'
        ]
    
    def start(self):
        try:
            self.playwright = sync_playwright().start()
            self.browser = self.playwright.chromium.launch(
                headless=self.headless,
                args=['--no-sandbox', '--disable-dev-shm-usage']
            )
            self.page = self.browser.new_page()
            self.page.set_viewport_size({"width": 1920, "height": 1080})
            print("✅ 브라우저 시작 완료")
            return True
        except Exception as e:
            print(f"❌ 브라우저 시작 실패: {e}")
            return False
    
    def check_place_tab(self, keyword):
        try:
            if not self.page:
                self.start()
            
            search_url = f"https://search.naver.com/search.naver?query={quote(keyword)}"
            self.page.goto(search_url, wait_until="domcontentloaded", timeout=10000)
            time.sleep(2)
            
            place_tab_selectors = [
                'a[data-tab="place"]',
                'a.tab[href*="place"]',
                '.api_subject_bx a[href*="place"]'
            ]
            
            has_place_tab = False
            for selector in place_tab_selectors:
                try:
                    element = self.page.query_selector(selector)
                    if element:
                        has_place_tab = True
                        break
                except:
                    continue
            
            return {
                'has_place_tab': has_place_tab,
                'confidence': 'high' if has_place_tab else 'low',
                'keyword': keyword,
                'message': f"✅ 플레이스 탭 표시됨" if has_place_tab else "❌ 플레이스 탭 없음"
            }
        except Exception as e:
            return {
                'has_place_tab': False,
                'confidence': 'error',
                'keyword': keyword,
                'message': f"오류: {str(e)}"
            }
    
    def search_places(self, keyword, max_results=20):
        try:
            if not self.page:
                self.start()
            
            print(f"🔍 '{keyword}' 검색 시작...")
            
            map_url = f"https://map.naver.com/p/search/{quote(keyword)}"
            self.page.goto(map_url, wait_until="domcontentloaded", timeout=15000)
            time.sleep(3)
            
            iframe = self.page.frame(name="searchIframe")
            if not iframe:
                print("❌ iframe을 찾을 수 없습니다")
                return []
            
            results = []
            
            for scroll_count in range(3):
                place_items = iframe.query_selector_all('li[role="listitem"]')
                
                for item in place_items:
                    if len(results) >= max_results:
                        break
                    
                    try:
                        place_data = self._extract_place_data(iframe, item, keyword)
                        if place_data and not self._is_duplicate(results, place_data):
                            results.append(place_data)
                    except:
                        continue
                
                if len(results) >= max_results:
                    break
                
                try:
                    scroll_area = iframe.query_selector('.Ryr1F')
                    if scroll_area:
                        iframe.evaluate('(element) => element.scrollTop += 1000', scroll_area)
                        time.sleep(1)
                except:
                    pass
            
            print(f"✅ {len(results)}개 결과 수집")
            return results
            
        except Exception as e:
            print(f"❌ 검색 오류: {e}")
            return []
    
    def _extract_place_data(self, iframe, item, keyword):
        try:
            name_elem = item.query_selector('.TYaxT, .place_bluelink, span.YwYLL')
            name = name_elem.inner_text().strip() if name_elem else ""
            
            if not name:
                return None
            
            category_elem = item.query_selector('.KCMnt, span.nQ2b9')
            category = category_elem.inner_text().strip() if category_elem else ""
            
            address_elem = item.query_selector('.LDgIH, .addr')
            address = address_elem.inner_text().strip() if address_elem else ""
            
            phone_elem = item.query_selector('.dry6Z, .tel')
            phone = phone_elem.inner_text().strip() if phone_elem else ""
            
            rating_elem = item.query_selector('.h69bs, .score')
            rating = rating_elem.inner_text().strip() if rating_elem else ""
            
            review_elem = item.query_selector('.Tvqnp, .cnt')
            reviews = review_elem.inner_text().strip() if review_elem else "0"
            reviews = re.sub(r'[^0-9]', '', reviews)
            
            is_other_region = self._detect_other_region_place(
                name, address, phone, rating, reviews, keyword
            )
            
            return {
                'name': name,
                'category': category,
                'address': address,
                'phone': phone,
                'rating': rating,
                'reviews': reviews,
                'is_other_region': is_other_region,
                'place_type': '타지역업체' if is_other_region else '주업체'
            }
        except:
            return None
    
    def _detect_other_region_place(self, name, address, phone, rating, reviews, keyword):
        indicators = 0
        
        if phone.startswith('070'):
            indicators += 3
        
        if address:
            address_parts = address.split()
            if len(address_parts) <= 3 or not any(char.isdigit() for char in address):
                indicators += 2
        
        if not rating or rating == "":
            indicators += 1
        
        if reviews == "0" or (reviews and int(reviews) < 3):
            indicators += 1
        
        if keyword:
            keyword_clean = re.sub(r'[^\w\s]', '', keyword)
            keyword_words = keyword_clean.split()
            if any(word in name for word in keyword_words if len(word) > 1):
                indicators += 2
        
        return indicators >= 4
    
    def _is_duplicate(self, results, new_place):
        for place in results:
            if place['name'] == new_place['name'] and place['address'] == new_place['address']:
                return True
        return False
    
    def close(self):
        try:
            if self.browser:
                self.browser.close()
            if self.playwright:
                self.playwright.stop()
        except:
            pass

print("✅ 크롤링 모듈 생성 완료")

## 4️⃣ Flask 웹 앱 생성

웹 인터페이스 및 API 서버 생성

In [ ]:
# Flask 앱 코드는 너무 길어서 파일로 생성
# 이전에 만든 naver_map_crawler.py를 사용
!echo "✅ Flask 앱 준비 완료"

## 5️⃣ 앱 실행 🚀

이 셀을 실행하면:
1. Flask 서버가 시작됩니다
2. ngrok 터널이 생성됩니다
3. **공개 URL**이 표시됩니다

⚠️ **주의**: 이 셀은 계속 실행됩니다. 중지하려면 "중지" 버튼을 클릭하세요.

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import threading
import os

# 코랩 환경에서 비동기 실행 허용
nest_asyncio.apply()

# Flask 앱 import
from flask import Flask, render_template, request, jsonify, send_file
import time
import csv
from datetime import datetime
from naver_place_crawler_real import RealNaverPlaceCrawler

# Flask 앱 생성 (간단 버전)
app = Flask(__name__)
crawler = None

@app.route('/')
def index():
    return '''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>네이버 플레이스 크롤링</title>
    <style>
        body { font-family: Arial; max-width: 1200px; margin: 50px auto; padding: 20px; }
        h1 { color: #667eea; }
        input { padding: 10px; width: 300px; font-size: 16px; }
        button { padding: 10px 20px; background: #667eea; color: white; border: none; cursor: pointer; font-size: 16px; }
        button:hover { background: #764ba2; }
        .result { margin-top: 30px; }
        .place-card { border: 2px solid #ddd; padding: 15px; margin: 10px 0; border-radius: 8px; }
        .other-region { border-color: #ff9800; background: #fff3e0; }
    </style>
</head>
<body>
    <h1>🗺️ 네이버 플레이스 크롤링 (구글 코랩)</h1>
    <p>검색어를 입력하고 검색 버튼을 클릭하세요.</p>
    
    <input type="text" id="keyword" placeholder="예: 강남역 맛집">
    <button onclick="search()">🔍 검색</button>
    
    <div id="result" class="result"></div>
    
    <script>
        async function search() {
            const keyword = document.getElementById('keyword').value;
            const resultDiv = document.getElementById('result');
            
            resultDiv.innerHTML = '<p>검색 중...</p>';
            
            try {
                const response = await fetch('/api/search', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ keyword, max_results: 20 })
                });
                
                const data = await response.json();
                
                if (data.success) {
                    let html = `<h2>검색 결과 (${data.count}개)</h2>`;
                    
                    data.results.forEach(place => {
                        const className = place.is_other_region ? 'place-card other-region' : 'place-card';
                        const badge = place.is_other_region ? '🟠 타지역업체' : '🟢 주업체';
                        
                        html += `
                            <div class="${className}">
                                <strong>${place.name}</strong> ${badge}<br>
                                📍 ${place.address}<br>
                                📞 ${place.phone}<br>
                                ${place.rating ? `⭐ ${place.rating} (리뷰 ${place.reviews}개)` : '평점 없음'}
                            </div>
                        `;
                    });
                    
                    resultDiv.innerHTML = html;
                } else {
                    resultDiv.innerHTML = `<p>오류: ${data.error}</p>`;
                }
            } catch (error) {
                resultDiv.innerHTML = `<p>오류: ${error.message}</p>`;
            }
        }
    </script>
</body>
</html>'''

@app.route('/api/search', methods=['POST'])
def api_search():
    global crawler
    
    try:
        data = request.json
        keyword = data.get('keyword', '')
        max_results = int(data.get('max_results', 20))
        
        if not keyword:
            return jsonify({'success': False, 'error': '검색어를 입력하세요'})
        
        if crawler is None:
            crawler = RealNaverPlaceCrawler(headless=True)
            crawler.start()
        
        results = crawler.search_places(keyword, max_results)
        
        return jsonify({
            'success': True,
            'results': results,
            'count': len(results)
        })
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

# Flask 서버를 별도 스레드에서 실행
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# ngrok 터널 생성
print("🚀 서버 시작 중...")
public_url = ngrok.connect(5000)
print("\n" + "="*70)
print("✅ 서버가 시작되었습니다!")
print("="*70)
print(f"\n🌐 접속 URL: {public_url}")
print("\n💡 위 URL을 클릭하거나 복사해서 브라우저에서 열어주세요!")
print("\n⚠️  주의: 이 셀이 실행 중일 때만 접속 가능합니다.")
print("\n" + "="*70 + "\n")

# Flask 서버 시작
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# 서버 유지
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n서버 종료")
    ngrok.disconnect(public_url)
    if crawler:
        crawler.close()

## 📖 사용 방법

1. **위 셀 실행 후 표시되는 ngrok URL 클릭**
2. 웹페이지에서 검색어 입력 (예: "강남역 맛집")
3. 검색 결과 확인:
   - 🟢 **주업체**: 일반 전화번호, 평점 있음
   - 🟠 **타지역업체**: 070번호, 평점 없음

---

## ⚠️ 주의사항

- 코랩 세션이 종료되면 서버도 종료됩니다
- 무료 ngrok은 2시간 제한이 있습니다
- 과도한 크롤링은 IP 차단의 원인이 될 수 있습니다

---

## 🎉 완료!

이제 실제 네이버 플레이스 데이터를 크롤링할 수 있습니다!